In [36]:
import sbibm
from sbi.inference import NPE, simulate_for_sbi
from sbi.utils import BoxUniform
import torch
import pickle
import sys  
sys.path.insert(1, '../')
from collective_posterior import CollectivePosterior

task = sbibm.get_task('gaussian_linear_uniform') # See sbibm.get_available_tasks() for all tasks
prior = task.get_prior()
simulator = task.get_simulator()
observation = task.get_observation(num_observation=1)  # 10 per task

# These objects can then be used for custom inference algorithms, e.g.
# we might want to generate simulations by sampling from prior:
thetas = prior(num_samples=100_000)
xs = simulator(thetas)
theta_dim = len(prior(num_samples=1)[0])
prior_min = torch.tensor([-1]*theta_dim, dtype=torch.float32)
prior_max = torch.tensor([1]*theta_dim, dtype=torch.float32)
inf_prior = BoxUniform(low=prior_min, high=prior_max)

simulator(torch.tensor([[ 1.0158, -0.7272, -0.5739,  0.5398,  0.2512, -0.3744,  0.7249,  0.9442,
          0.2100, -0.5636]]))

tensor([[ 1.1276, -0.9320, -0.2715,  0.6293,  0.2709, -0.1716,  0.6952,  0.8199,
          0.3477, -0.4175]])

In [2]:
inference = NPE(prior=inf_prior)

density_estimator = inference.append_simulations(thetas, xs).train()
posterior = inference.build_posterior(density_estimator)

# Save the posterior with pickle
with open(f'posterior_GLU.pkl', 'wb') as f:
    pickle.dump(posterior, f)


2024-09-18 15:55:01.456235: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 15:55:02.241940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyboardInterrupt: 

In [ ]:
posterior = pickle.load(open('posterior_GLU.pkl','rb'))
theta = prior()
obs = simulator(theta)
m = posterior.set_default_x(obs).sample((100,)).mean(0)

In [ ]:
# Evaluate accuracy
n = 100
theta = prior(n)
obs = simulator(theta)
res = torch.empty(n,theta_dim)

for i in range(n):
    m = posterior.set_default_x(obs[i]).sample((200,)).mean(0)
    res[i,:] = m-theta[i]

In [ ]:
import matplotlib.pyplot as plt
plt.hist(res[:,0])

In [ ]:
theta = prior()
X = torch.empty((10,len(observation[0])))
for i in range(10):
    X[i] = simulator(theta)[0]

In [ ]:
from collective_posterior import CollectivePosterior

n_eval = 300 # N used to calculate normalizng constant
epsilon = -150 # choice is detailed later
op = CollectivePosterior(inf_prior, posterior, X, n_eval, log_C=1, epsilon=-500)
print(op.get_log_C())

In [ ]:
y = torch.abs(op.sample(1000).mean(0)-theta)